In [ ]:
from pprint import pprint
import copy
import types
from jsonpath_ng.ext import parse


class Chainable:
    pass


class ChainableContext:
    def __init__(self, dryrun, debug):
        self.dryrun = dryrun
        self.debug = debug
        self.workflow = []

    def add_mapping(self, mapping):
        self.workflow.append(mapping)

    def run(self):
        return


class ChainableObject:
    def __init__(self, context=None):
        self.content = {
            'data': {},
            'meta': {},
            'hist': [],
            'map': {}
        }

    def resolve(self, mapping):
        #make components availabel in function scobe
        data = self.content['data']
        meta = self.content['meta']
        hist = self.content['hist']
        for key in mapping['param']:
            res = []
            value = mapping['param'][key]
            if isinstance(value, dict): #dynamic
                if 'static' in value: res.append(value['static']) #explicit static 
                if 'eval' in value: res.append(eval(value['eval'])) #eval expression
                if 'jsonpath' in value:
                    jsonpath_expr = parse(value['jsonpath'])
                    for match in jsonpath_expr.find(self.content):
                        res.append(match.value)
                if 'match' in value:  #match condition
                    if 'meta' in value['match']:
                        jsonpath_expr = parse(value['match']['meta']['jsonpath'])
                        #[print(str(match.full_path)) for match in jsonpath_expr.find(self.content)]
                        #[pprint(match.value) for match in jsonpath_expr.find(obj.content)]
                        for match in jsonpath_expr.find(self.content):
                            data_path = str(match.full_path).replace('meta', 'data', 1) #default: replace only root key => traverse to data branch
                            if 'value' in value and 'data' in value['value']:
                                data_path = str(match.full_path).replace('meta', 'data', 1) + "." + value['value']['data']['jsonpath'] #value path relative to match path
                            if 'value' in value and 'meta' in value['value']:
                                data_path = str(match.full_path) + "." + value['value']['meta']['jsonpath'] #value path relative to match path
                            print(data_path)
                            #pprint(parse(data_path).find(self.content)[0].value)
                            res.append(parse(data_path).find(self.content)[0].value)
                if (len(res) == 1): res = res[0]
                mapping['param'][key] = res
            if isinstance(value, types.FunctionType): #dynamic function
                mapping['param'][key] = value()
        return mapping
        
    def apply(self, mapping):
        if mapping is None:
            mapping = self.content['map'] #read map from object (result from previous function)
        self.content = eval(mapping['func'] + ".apply(self.content, mapping['param'])")
        #raw_data(self.content, mapping)
        #pprint(mapping)
        #pprint(self.content)
        return self

l1 = lambda: False
obj = ChainableObject()
obj.apply({
    'func': "get_raw",
    'param': {'debug': False}
})
pprint(obj.content)
mapping = { 'param': {
    'debug1': True,
    'debug2' : lambda: False,
    'debug3': {'static': False},
    'debug4': {'eval': "hist[-1]['func']['name'] == 'get_raw'"},
    'param1': {'eval': "data['raw'][0]"},
    'param2': {'jsonpath': 'meta.*[?name = "raw"].label.de'}, #eval jsonpath
    'param3': {'match': {'meta': {'jsonpath': 'meta.*[?name = "raw"]'}}, 'value': {'data': {'jsonpath': '[0]'}}}, #default: traverse to data branch
    'param4': {'match': {'meta': {'jsonpath': 'meta.*[?name = "raw"]'}}, 'value': {'meta': {'jsonpath': 'label'}}}, #value path relative to match path
}}
mapping = obj.resolve(mapping)
pprint(mapping)
    
def get_mapping():
    return {
        'func': "Chainable.get_raw",
        'param': {'debug': {'static': False}}
    }
      
Chainable.get_raw = GetRaw()
obj = ChainableObject()
obj.apply({
    'func': "Chainable.get_raw",
    'param': {'debug': False}
}).apply(get_mapping())
pprint(obj.content)

workflow = [{
    'func': "Chainable.get_raw",
    'param': {'debug': False}
},{
    'func': "Chainable.get_raw",
    'param': {'debug': False, 'data_name': "RawVoltage"}
}]
obj2 = ChainableObject()
for step in workflow:
    obj2 = obj2.apply(step)
pprint(obj2.content)



In [3]:
from abc import ABCMeta, abstractmethod

class ChainableFunctionParam(BaseModel)
    param_dict: dict

class ChainableFunction:
    def __init__(self, name: str, uuid: str):
        self.name = name
        self.uuid = uuid
    
    def set_default_params(self, param: dict):
        self.param_default = param 
    
    def set_param(self, param: dict = None):        
        if not param is None:
            param = {**self.param_default, **param}
        else: param = self.param_default
        pprint(param)
        return param
    
    def apply(self, obj: dict, param: dict = None):
        param = self.set_param(param)
        obj = self.pre_exec(obj, param)
        obj = self.func(obj, param)
        obj = self.post_exec(obj, param)
        return obj
    
    def pre_exec(self, obj, param):
        hist = {}
        if param['debug']: hist['data'] = copy.deepcopy(obj['data'])
        if param['debug']: hist['map'] = copy.deepcopy(obj['map'])
        hist['func'] = {'func_id': self.uuid, 'name': "get_raw", 'time':"...", 'param':param}
        obj['hist'].append(hist)
        return obj
    
    @abstractmethod
    def func(obj, param):
        pass
    
    def store_data(self, obj, key, data, meta):
        if not key in obj['data']: obj['data'][key] = []
        obj['data'][key].append(data)
        if not key in obj['meta']: obj['meta'][key] = []
        obj['meta'][key].append(meta)
        
    def post_exec(self, obj, param):

        return obj
    
class GetRaw(ChainableFunction):
    def __init__(self):
        super().__init__("get_raw", "0001")
        super().set_default_params({'debug': True, 'data_name':"raw"})

    def func(self, obj, param):
        
        super().store_data(obj, 'raw', [1,2,3,4], {'type': "list", 'dim': 1, 'name': param['data_name'], 'label': {'de':"Spannung"}, 'quant': "qudt:Voltage", 'unit':"qudt:mV", 'test':{'nested': "value"}})

        return obj
    

#GetRaw.name
get_raw = GetRaw()
obj = ChainableObject()
obj = get_raw.apply(obj.content)
pprint(obj)

{'data_name': 'raw', 'debug': True}
{'data': {'raw': [[1, 2, 3, 4]]},
 'hist': [{'data': {},
           'func': {'func_id': '0001',
                    'name': 'get_raw',
                    'param': {'data_name': 'raw', 'debug': True},
                    'time': '...'},
           'map': {}}],
 'map': {},
 'meta': {'raw': [{'dim': 1,
                   'label': {'de': 'Spannung'},
                   'name': 'raw',
                   'quant': 'qudt:Voltage',
                   'test': {'nested': 'value'},
                   'type': 'list',
                   'unit': 'qudt:mV'}]}}


In [42]:
from abc import ABCMeta, abstractmethod
from pydantic import BaseModel
from typing import List, Optional
#from dataclass_wizard import JSONWizard
#from dataclass_wizard import fromdict, asdict
#from dataclasses import dataclass
#from pydantic.dataclasses import dataclass

class Data(BaseModel):
#@dataclass
#class Data(BaseModel, JSONWizard):
#@dataclass
#class Data(JSONWizard):
    id: int
    ks: str
    items: List[str]
    sub: Optional[Data] = None

data_dict = {'id': 1, 'ks': 'test', 'items': ['1', '2', '3'], 'sub': {'id': '1', 'ks': 'test', 'items': ['1']}}
data = Data(**data_dict)
#data = fromdict(Data, data_dict)
#print(data.id)
#print(data.sub.id)
print(repr(data))
data.sub.items

Data(id=1, ks='test', items=['1', '2', '3'], sub=Data(id=1, ks='test', items=['1'], sub=None))


In [68]:
from abc import ABCMeta, abstractmethod
from pydantic import BaseModel
from typing import List, Optional

class Function:
    #@abstractmethod  
    def __init__(self):
        print("Function")
        #pass

    @abstractmethod
    def func(self, param: dict):
        pass

class TypeSafeFunction(Function):
    class Param(BaseModel):
        pass
    param_class: type(TypeSafeFunction.Param)    
    def __init__(self, param_class: type(TypeSafeFunction.Param)):
    #param_class: Param  
    #def __init__(self, param_class: Param):
        #super(TypeSafeFunction, self).__init__(param_class=param_class)
        assert issubclass(param_class, TypeSafeFunction.Param)
        print(type(TypeSafeFunction.Param))
        #super().__init__(param_class=param_class)
        super().__init__(param_class=param_class)

    def apply(self, **kwargs):
        p = self.param_class(**kwargs)
        print(type(p))
        self.func(p)
    
    @abstractmethod
    def func(self, param: Param):
        pass
    
class B():

    def func(self, param):
        self.param = param

class C(TypeSafeFunction):
    class Param():#TypeSafeFunction.Param):
        debug: int

    def __init__(self):
        super().__init__(param_class=C.Param)
        
    def func(self, param: C.Param):
        print(repr(param))

p_dict = {'debug':'1', 'data_name': "test"}
#p = Function.Param(**p_dict)
b = B()
b.func(p_dict)
#print(b.param)
c = C()
#c = C(param_class=B.Param)
c.apply(**p_dict)
#print(repr(c.param))

AssertionError: 